# **API & WEBHOOK-BASED DATA RETRIEVAL**

### **1) REQUIREMENTS SETUP**

In [11]:
# !pip install -r requirements.txt

In [63]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np

### **2) MODULES IMPORT**

In [29]:
from FRED_module import fetch_FRED
from EUROSTAT_module import fetch_EUROSTAT
from WB_module import fetch_WB 
from YFINANCE_module import fetch_YFINANCE
from DBNOMICS_module import fetch_DBNOMICS

### **3) DATA FETCHING**

#### **3.1) EUROSTAT-extracted indicators**

In [ ]:
# Export to the US by EU member state (monthly, total, millions of EUR)
# EU27 (from 2020) trade by SITC product group (Trade value in million ECU/EURO, Standard International Trade Classification (SITC Rev. 4, 2006): Total - all products)
# https://ec.europa.eu/eurostat/databrowser/view/ds-059331/legacyMultiFreq/table?lang=en 

# EURO_expus_m_raw = 

In [ ]:
# Industrial Production by EU member state (monthly, total manufacturing, Index: 2021=100, non-seasonally adjusted, 1996-01, 2025-08)
# https://ec.europa.eu/eurostat/databrowser/view/sts_inpr_m/default/table?lang=en
EURO_indprod_m_raw = fetch_EUROSTAT(
    "sts_inpr_m",
    filters = {
    "geo": ["EU27_2020", "EU28", "EA20", "EA19",  
    "BE", "BG", "CZ", "DK", "DE", "EE", "IE", "EL", "ES", "FR",
    "HR", "IT", "CY", "LV", "LT", "LU", "HU", "MT", "NL", "AT",
    "PL", "PT", "RO", "SI", "SK", "FI", "SE"],
    "s_adj": "NSA",
    "unit": "I21",
    "nace_r2": "C"
    }
)

EURO_indprod_m_raw = EURO_indprod_m_raw[[ "geo", "time", "value" ]]
EURO_indprod_m_raw = EURO_indprod_m_raw.rename(columns={
    "geo": "Country",
    "time": "Time",
    "value": "Indprod Index (I21)"
})

EURO_indprod_m_raw.tail()

,Country,Time,Indprod Index (I21)
8971,SK,2025-04,102.5
8972,SK,2025-05,101.8
8973,SK,2025-06,100.2
8974,SK,2025-07,87.7
8975,SK,2025-08,86.4


In [58]:
# Unemployment rate by EU member state (monthly, Percentage of population in the labour force (age-class = total, sex-class=total), non-seasonally adjusted, 1994-01 (varying), 2025-08)
# https://ec.europa.eu/eurostat/databrowser/view/une_rt_m/default/table?lang=en
EURO_unem_m_raw = fetch_EUROSTAT(
    "une_rt_m",
    filters = {
    "geo": ["EU27_2020", "EU28", "EA20", "EA19",  
    "BE", "BG", "CZ", "DK", "DE", "EE", "IE", "EL", "ES", "FR",
    "HR", "IT", "CY", "LV", "LT", "LU", "HU", "MT", "NL", "AT",
    "PL", "PT", "RO", "SI", "SK", "FI", "SE"],
    "s_adj": "NSA",
    "unit": "PC_ACT"
    }
)

EURO_unem_m_raw = EURO_unem_m_raw[[ "geo", "time", "value" ]]
EURO_unem_m_raw = EURO_unem_m_raw.rename(columns={
    "geo": "Country",
    "time": "Time",
    "value": "Unemployment Rate (%pop in LF)"
})

EURO_unem_m_raw.tail()

,Country,Time,Unemployment Rate (%pop in LF)
98345,SK,2025-04,13.9
98346,SK,2025-05,14.3
98347,SK,2025-06,15.4
98348,SK,2025-07,15.9
98349,SK,2025-08,15.8


In [ ]:
# US Unemployment rate (monthly, Percentage of population in the labour force (age-class = total, sex-class=total), non-seasonally adjusted, 1983-01, 2025-08)
# https://ec.europa.eu/eurostat/databrowser/view/une_rt_m/default/table?lang=en
US_unem_m_raw = fetch_EUROSTAT(
    "une_rt_m",
    filters = {
    "geo": "US",
    "s_adj": "NSA",
    "unit": "PC_ACT"
    }
)

US_unem_m_raw = US_unem_m_raw[[ "geo", "time", "value" ]]
US_unem_m_raw = US_unem_m_raw.rename(columns={
    "geo": "Country",
    "time": "Time",
    "value": "Unemployment Rate (%pop in LF)"
})

US_unem_m_raw.tail()

,Country,Time,Unemployment Rate (%pop in LF)
4603,US,2025-04,8.5
4604,US,2025-05,9.7
4605,US,2025-06,11.4
4606,US,2025-07,10.8
4607,US,2025-08,10.7


#### **3.2) FRED-extracted indicators**

In [71]:
# US-EUR exchange rate, U.S. Dollars to One Euro (initially daily, converted to monthly, non-seasonally adjusted, 1999-01, 2025-10)
# https://fred.stlouisfed.org/series/DEXUSEU
EXUSEU_d_raw = fetch_FRED("DEXUSEU") 
EXUSEU_d_raw = EXUSEU_d_raw.rename(columns= 
        {"date": "Time", 
         "DEXUSEU": "USD-EUR Spot Exchange Rate"
})

# Aggregation (dimension from daily to monthly)
EXUSEU_d_raw = EXUSEU_d_raw.set_index("Time")
EXUSEU_m_raw = EXUSEU_d_raw.resample("M").mean().reset_index()
EXUSEU_m_raw["Time"] = EXUSEU_m_raw["Time"].dt.to_period("M").astype(str)

EXUSEU_m_raw.tail()

,Time,USD-EUR Spot Exchange Rate
317,2025-06,1.153365
318,2025-07,1.167082
319,2025-08,1.164748
320,2025-09,1.173871
321,2025-10,1.164894


In [77]:
# US-BLEV exchange rate, U.S. Dollars to One Bulgarian Lev (monthly, non-seasonally adjusted, 1960-01, 2021-06)
# https://fred.stlouisfed.org/series/BGRCCUSMA02STM

EXUSBLEV_m_raw = fetch_FRED("BGRCCUSMA02STM") 
EXUSBLEV_m_raw = EXUSBLEV_m_raw.rename(columns= 
        {"date": "Time", 
         "BGRCCUSMA02STM": "USD-BLEV Spot Exchange Rate"
})

EXUSBLEV_m_raw["Time"] = EXUSBLEV_m_raw["Time"].dt.to_period("M").astype(str)

EXUSBLEV_m_raw.tail()

,Time,USD-BLEV Spot Exchange Rate
769,2021-02,1.6167
770,2021-03,1.6447
771,2021-04,1.6366
772,2021-05,1.6084
773,2021-06,1.6237


In [85]:
# US-SD exchange rate, U.S. Dollar to One Swedish Kronor (initially SD-US daily, converted to US-SD monthly, non-seasonally adjusted, 1971-01, 2025-10)
# https://fred.stlouisfed.org/series/DEXSDUS
EXSDUS_d_raw = fetch_FRED("DEXSDUS") 
EXSDUS_d_raw = EXSDUS_d_raw.rename(columns= 
        {"date": "Time", 
         "DEXSDUS": "SD-USD Spot Exchange Rate"
})

# Aggregation (dimension from daily to monthly)
EXSDUS_d_raw = EXSDUS_d_raw.set_index("Time")
EXSDUS_m_raw = EXSDUS_d_raw.resample("M").mean().reset_index()
EXSDUS_m_raw["Time"] = EXSDUS_m_raw["Time"].dt.to_period("M").astype(str)

# Conversion to USXS spot exchange rate 
EXUSSD_m_raw = EXSDUS_m_raw.copy()
EXUSSD_m_raw["USD-SD Spot Exchange Rate"] = 1 / EXUSSD_m_raw["SD-USD Spot Exchange Rate"]
EXUSSD_m_raw = EXUSSD_m_raw.drop(columns=["SD-USD Spot Exchange Rate"])

EXUSSD_m_raw.tail()

,Time,USD-SD Spot Exchange Rate
653,2025-06,0.104732
654,2025-07,0.104194
655,2025-08,0.104412
656,2025-09,0.106693
657,2025-10,0.106036


In [ ]:
# US-DN exchange rate, U.S. Dollar to One Danish Krone (initially DN-US converted to US-DN, monthly non-seasonally adjusted, 1971-01, 2025-09)
# https://fred.stlouisfed.org/series/EXDNUS
EXDNUS_m_raw = fetch_FRED("EXDNUS") 
EXDNUS_m_raw = EXDNUS_m_raw.rename(columns= 
        {"date": "Time", 
         "EXDNUS": "DN-USD Spot Exchange Rate"
})

EXDNUS_m_raw["Time"] = EXDNUS_m_raw["Time"].dt.to_period("M").astype(str)

# Conversion to USDN spot exchange rate 
EXUSDN_m_raw = EXDNUS_m_raw.copy()
EXUSDN_m_raw["USD-DN Spot Exchange Rate"] = 1 / EXUSDN_m_raw["DN-USD Spot Exchange Rate"]
EXUSDN_m_raw = EXUSDN_m_raw.drop(columns=["DN-USD Spot Exchange Rate"])

EXUSDN_m_raw.head()

,Time,USD-DN Spot Exchange Rate
0,1971-01,0.133608
1,1971-02,0.133593
2,1971-03,0.133676
3,1971-04,0.133535
4,1971-05,0.133337


#### **3.5) OTHERS-extracted indicators**

In [ ]:
# Tariffs ratio